### Torrent to Google Drive Downloader - Seedr Style

#### ==== USER CONFIGURATION ====
##### Examples:
 - .torrent file URL: "https://yts.mx/torrent/download/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
 - Magnet link: "magnet:?xt=urn:btih:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
TORRENT_LINK = "https://yts.mx/torrent/download/A87D36A8EFD2924E9FB607722DE5B453C625884A"

In [ ]:
while True:
    TORRENT_LINK = input("Enter Torrent/Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break

In [ ]:
# ==== INSTALL DEPENDENCIES ====
!apt install -y python3-libtorrent
!pip install ipywidgets requests

import libtorrent as lt
import time
import os
import requests
from IPython.display import display
import ipywidgets as widgets
from google.colab import drive

# ==== MOUNT GOOGLE DRIVE ====
drive.mount("/content/drive")
BASE_PATH = "/content/drive/My Drive/Torrent"
os.makedirs(BASE_PATH, exist_ok=True)

# ==== INITIALIZE LIBTORRENT SESSION ====
ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()
ses.start_lsd()
ses.start_upnp()
ses.start_natpmp()

downloads = []

# ==== FUNCTION TO ADD TORRENT ====
def add_torrent_from_file(file_path):
    info = lt.torrent_info(file_path)
    save_path = f"{BASE_PATH}/{info.name()}"
    os.makedirs(save_path, exist_ok=True)
    params = {"save_path": save_path, "ti": info}
    downloads.append(ses.add_torrent(params))
    print(f"📥 Added torrent: {info.name()} -> {save_path}")

def add_torrent_from_url(torrent_url):
    local_file = "/content/temp.torrent"
    r = requests.get(torrent_url, allow_redirects=True)
    with open(local_file, "wb") as f:
        f.write(r.content)
    add_torrent_from_file(local_file)

def add_torrent_from_magnet(magnet_link):
    params = {"save_path": BASE_PATH}
    h = lt.add_magnet_uri(ses, magnet_link, params)
    downloads.append(h)
    print(f"🔗 Added magnet link")

# ==== DETECT LINK TYPE ====
if TORRENT_LINK.lower().startswith("magnet:"):
    add_torrent_from_magnet(TORRENT_LINK)
elif TORRENT_LINK.lower().endswith(".torrent") or "yts.mx/torrent/download" in TORRENT_LINK:
    add_torrent_from_url(TORRENT_LINK)
else:
    raise ValueError("❌ Invalid TORRENT_LINK. Must be a magnet link or direct .torrent file URL.")

# ==== PROGRESS BAR WITH ETA ====
state_str = [
    "queued", "checking", "downloading metadata",
    "downloading", "finished", "seeding",
    "allocating", "checking fastresume"
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(step=0.01, disabled=True, layout=layout, style=style)
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
            eta = (s.total_wanted - s.total_done) / s.download_rate if s.download_rate > 0 else 0
            bar.description = f"{download.name()} | {s.download_rate/1000:.1f} kB/s | {s.progress*100:.1f}% | ETA: {eta/60:.1f} min | {state_str[s.state]}"
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()
            download_bars.remove(bar)
            print(f"✅ {download.name()} complete")
    time.sleep(1)

# ==== AUTO-UNMOUNT GOOGLE DRIVE ====
drive.flush_and_unmount()
print("🎉 All torrents completed and Google Drive unmounted.")